<a href="https://colab.research.google.com/github/leeds1219/DL_and_AI_Notes_and_Projects/blob/main/ML_Falcon_40B_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu111/torch_stable.html --upgrade

Looking in indexes: https://download.pytorch.org/whl/cu111/torch_stable.html


In [ ]:
!pip install langchain einops accelerate transformers bitsandbytes

In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import os
import torch

In [ ]:
torch.cuda.is_available()

True

# Pipeline

In [ ]:
# Define Model ID
model_id = "tiiuae/falcon-40b-instruct"
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Load Model
model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir='./workspace/',
    torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", offload_folder="offload")
# Set PT model to inference mode
model.eval()
# Build HF Transformers pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
# Test out the pipeline
pipeline('who is spongebob?')

# Langchain

In [ ]:
# Setup prompt template
template = PromptTemplate(input_variables=['input'], template='{input}')
# Pass hugging face pipeline to langchain class
llm = HuggingFacePipeline(pipeline=pipeline)
# Build stacked LLM chain i.e. prompt-formatting + LLM
chain = LLMChain(llm=llm, prompt=template)

In [ ]:
# Test LLMChain
response = chain.run('who is spongebob?')

# Gradio App

In [ ]:
# Install Gradio for the UI component
!pip install gradio

In [ ]:
# Create generate function - this will be called when a user runs the gradio app
def generate(prompt):
    # The prompt will get passed to the LLM Chain!
    return chain.run(prompt)
    # And will return responses

In [ ]:
# Define a string variable to hold the title of the app
title = '🦜🔗 Falcon-40b-Instruct'
# Define another string variable to hold the description of the app
description = 'This application demonstrates the use of the open-source `Falcon-40b-Instruct` LLM.'
# pls subscribe 🙏

In [ ]:
# Build gradio interface, define inputs and outputs...just text in this
gr.Interface(fn=generate, inputs=["text"], outputs=["text"],
             # Pass through title and description
             title=title, description=description,
             # Set theme and launch parameters
             theme='finlaymacklon/boxy_violet').launch(server_port=8080, share=True)